In [10]:
# disable future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# safe imports
import os, sys       # arcgis comes with these
import datetime      # arcgis comes with this
import numpy as np   # arcgis comes with this
import pandas as pd  # arcgis comes with this
import matplotlib.pyplot as plt

# risk imports (non-native to arcgis)
try:
    import xarray as xr  # not in arcgis
except:
    arcpy.AddError('Python library Xarray is not installed.')
    raise

# import tools
try:
    # shared folder
    sys.path.append(r'C:\Users\262272G\Documents\GitHub\tenement-tools\shared')
    sys.path.append(r'C:\Users\Lewis\Documents\GitHub\tenement-tools\shared')
    import arc, satfetcher, tools  

    # module folder
    sys.path.append(r'C:\Users\262272G\Documents\GitHub\tenement-tools\modules')
    sys.path.append(r'C:\Users\Lewis\Documents\GitHub\tenement-tools\modules')
    import ensemble, cog  

except:
    arcpy.AddError('Could not find tenement tools python scripts (modules, shared).')
    raise

In [19]:
out_nc = r"C:\Users\262272G\Desktop\test\cva5.nc"
ds = xr.open_dataset(out_nc)
ds = ds.compute()

ds = ds.drop(['angle_all', 'magnitude_all'])

In [21]:
# for current project, open current map
aprx = arcpy.mp.ArcGISProject('CURRENT')
m = aprx.activeMap

# create output folder using datetime as name
dt = datetime.datetime.now().strftime('%d%m%Y%H%M%S')
out_folder = os.path.join(os.path.dirname(out_nc), 'cva' + '_' + dt)
os.makedirs(out_folder)                

# disable visualise on map temporarily
arcpy.env.addOutputsToMap = False

# create crf filename and copy it
out_file = os.path.join(out_folder, 'cva.crf')
crf = arcpy.CopyRaster_management(in_raster=out_nc, 
                                  out_rasterdataset=out_file)

# add to map                  
m.addDataFromPath(crf) 

# re-enable add to map
arcpy.env.addOutputsToMap = True

# tailor cmap depending on method and parameters
#if in_trend_method == 'Mann-Kendall':
    #if mk_dir == 'both':
        #cmap = 'Red-Blue (Continuous)'
    #elif mk_dir == 'inc':
        #cmap = 'Yellow-Green-Blue (Continuous)'
    #else:
        #cmap = 'Yellow-Orange-Red (Continuous)'   
#else:
    #cmap = 'Red-Blue (Continuous)'

# apply cmap
#lyr = arc.apply_cmap(aprx=aprx, 
                     #lyr_name='trend.crf', 
                     #cmap_name=cmap, 
                     #cutoff_pct=0.0001) 

# invert if mk dec
#if in_trend_method == 'Mann-Kendall' and mk_dir == 'dec':
    #lyr.invertColorRamp = True